# Homework 5: Bootstrap, Resampling, CLT

Deadline:

This assignment is due on February 17 (Friday), 2023 at 23:59

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged. Refer to the policies page to learn more about how to learn cooperatively.

You should start early so that you have time to get help if you're stuck.


In [ ]:
# Don't change this cell; just run it. 
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.",
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

## 1. Preliminaries

Siméon has collected some data about the amount of time he has to wait in between every encounter of a wild Pokemon in the wild grass of a given area. He wants to estimate the variance `V` of waiting times in the population from the sample he has collected.

We normally investigate the random variation among our estimates by simulating a sampling procedure from the population many times and computing estimates from each sample that we generate.  In real life, if Siméon had known what the population looked like, he would have known `V` and would not have had any reason to think about random sampling. However, he didn't know what the population looked like, so he couldn't have run the simulations that we normally do. 

Simulating a sampling procedure many times was a useful exercise in *understanding random variation* for an estimate, but it's not as useful as a tool for practical data analysis.

Let's flip that sampling idea on its head to make it practical. **Given *just* a random sample of waiting times, we'll estimate `V`, and then we'll use simulation to find out how accurate our estimate probably is, without ever looking at the whole population.**  This is an example of *statistical inference*.

We (pokemon trainers/Siméon) want to know the variance of the waiting times between encountering wild Pokemon. That number is `V`.

We only see a small number of waiting times (assumed to be a random sample with replacement from among all the waiting times), so we have to use estimation.

#### Question 1.1
Is `V` a population parameter or a statistic?  If we use our random sample to compute a number that is an estimate of `V`, is that a population parameter or a statistic?

Set `V` and `V_estimate` to either the string `"parameter"` or `"statistic"` to indicate whether each value is a parameter or a statistic.

<!--
BEGIN QUESTION
name: q1_1
manual: false
-->

In [ ]:
V = ...
V_estimate = ...

To make the situation realistic, we're going to hide the variance of waiting times from you.  You'll have access only to this random sample:

In [ ]:
observations = Table.read_table("waiting_times.csv")
observations

#### Question 1.2
Logically, it makes sense to estimate the variance of the population using the variance of our sample. Write a function that computes that statistic.  It should take as its argument an array of waiting times and return the variance of them.  Call the function `var_based_estimator`.  

After that, use the function and the `observations` table to compute an estimate of `V` called `var_based_estimate`.

As a reminder, the variance of a distribution is the average of the deviations from mean squared.

$$ \text{average of} [(values - mean(values))^2]$$

<!--
BEGIN QUESTION
name: q1_2
manual: false
-->

In [ ]:
def var_based_estimator(nums):
    mean = ...
    deviations = ...
    squared_deviations = ...
    variance = ...
    return variance

var_based_estimate = var_based_estimator(observations.column(0))
var_based_estimate

#### Question 1.3
We want to try some different estimators so that we can compare our results to the `var_based_estimator` and see if they're similar. Our friend Gregory is an expert Pokémon player and actually knows what the true distribution is. He gives us a suggestion to try estimating the variance **using the mean**. Although this seems counterintuitive, let's try it out. 

We'll try to estimate `V` by using the mean of values in the sample.  Compute this value using the `observations` table and give it the name `mean_estimate`.

<!--
BEGIN QUESTION
name: q1_3
manual: false
-->

In [ ]:
mean_estimate = ...
mean_estimate

We can't just confidently proclaim that `mean_estimate` or `var_based_estimate` is equal to `V`.  What if we're really far off?  We want to get a sense of the accuracy of our estimates, which is what we'll do in the section below.

## 2. Resampling
To do this, we'll use resampling.  That is, we won't draw new samples from our population.  Rather we sample from our current sample, or "resample."

Why does that make sense?

When we try to find the value of a population parameter, we ideally would like to use the whole population. However, we often only have access to one sample and we must use that to estimate the parameter instead.

Here, we would like to use the population of waiting times to draw more samples and run a simulation about estimates of `V`.  But we still only have our sample.  So, we **use our sample in place of the population** to run the simulation. We resample from our original sample with replacement as many times as there are elements in the original sample. This resampling technique is called *bootstrapping*. 

Note that in order for bootstrapping to work well, you must start with a large, random sample. Then the Law of Large Numbers says that with high probability, your sample is representative of the population.

#### Question 2.1
Write a function called `simulate_resample`. The function should take one argument `tbl`, which is a table like `observations`. The function should generate and return a resample from the observed waiting times in `tbl`.

<!--
BEGIN QUESTION
name: q2_1
manual: false
-->

In [ ]:
def simulate_resample(tbl):
    ...

simulate_resample(observations) # Don't delete this line

We'll use many resamples at once to see what estimates typically look like.  However, we don't often pay attention to single resamples, so it's easy to misunderstand them.  Let's first answer some questions about our resample.

#### Question 2.2
Which of the following statements are true?

1. The resample can contain waiting times that are not in the original sample.
2. The resample has either zero, one, or more than one copy of each waiting time.
3. The original sample can contain waiting times that are not in the resample.
4. Each observed waiting time occurs exactly once in the original sample.
5. The resample has exactly the same sample size as the original sample.
6. The histogram of a resample can look somewhat different from the histogram of the original sample.

Assign `true_statements` to an **array** of the number(s) corresponding to correct statements.

*Note:* The "original sample" refers to `observations`, and the "resample" refers the output of one call of `simulate_resample()`. 

<!--
BEGIN QUESTION
name: q2_2
manual: false
-->

In [ ]:
true_statements = ...

Now let's write a function to do many resamples at once.

#### Question 2.3
Write a function called `sample_estimates`.  It should take 3 arguments:
1. `waiting_times_tbl`: A table from which the data should be sampled.  The table will look like `observations`. 
2. `statistic`: A *function* that takes in an array of waiting times as its argument and computes a statistic from the array (i.e. returns a calculated number). 
3. `repetitions`: The number of simulations to perform.

*Hint: You should use the function `simulate_resample` which you defined in Question 2.1*

The function should simulate `repetitions` number of samples **with replacement** from the given table. For each of those samples, it should compute the statistic on that sample. Then it should **return an array** containing each of those statistics.  The code below provides an example use of your function and describes how you can verify that you've written it correctly.

<!--
BEGIN QUESTION
name: q2_3
manual: false
-->

In [ ]:
def sample_estimates(waiting_times_tbl, statistic, repetitions):
    ...

# DON'T CHANGE THE CODE BELOW THIS COMMENT! 
# This is just an example to test your function.
# This should generate an empirical histogram of var-based estimates
# of V from samples of size 200 if V is 15.  This should be very roughly bell shaped and centered at about 14-15
# answer, make sure that's what you see!
num_repetitions = 500
population = Table.read_table("population_waiting_times.csv")
one_sample = Table.read_table("one_sample.csv") #This is a sample from the population table
example_estimates = sample_estimates(
    one_sample,
    var_based_estimator,
    num_repetitions)
Table().with_column("variance-based estimate", example_estimates).hist()

Now we can go back to the sample we actually observed (the table `observations`) and estimate how much our var-based estimate of `V` would have varied from sample to sample.

#### Question 2.4
Using the bootstrap and the sample `observations`, simulate the approximate distribution of *var-based estimates* of `V`.  Use 8000 replications and save the estimates in an array called `bootstrap_var_based_estimates`.  

We have provided code that plots a histogram, allowing you to visualize the simulated estimates.

*Hint:* What function did you previously define to compute variance-based estimates?

<!--
BEGIN QUESTION
name: q2_4
manual: false
-->

In [ ]:
bootstrap_var_based_estimates = ...

# Don't change the code below! This plots bootstrap_mean_based_estimates.
Table().with_column("var-based estimate", bootstrap_var_based_estimates).hist(bins=10) 

#### Question 2.5
Using the bootstrap and the sample `observations`, simulate the approximate distribution of *mean estimates* of `V`.  Use 8000 replications and save the estimates in an array called `bootstrap_mean_estimates`.

We have provided code that plots a histogram, allowing you to visualize the simulated estimates.

*Hint:* Similar to the question above, you should pass in a function as your second argument. In this case, you want **mean** estimates.

<!--
BEGIN QUESTION
name: q2_5
manual: false
-->

In [ ]:
bootstrap_mean_estimates = ...

# Don't change the code below! This plots bootstrap_max_estimates.
Table().with_column("mean estimate of the variance", bootstrap_mean_estimates).hist(bins=10) 

#### Question 2.6
`V` was actually 8! Compare the histograms of estimates you generated in 2.4 and 2.5 and answer the following questions:

1. How does the distribution of values for the mean-based estimates differ from the variance based estimates? Do both distributions contain the true variance value?
2. Which estimator do you think will work well for other distributions? Do you think the mean will always be a good estimate for the variance?

*Hint: Is the mean always equal to the variance?*

<!--
BEGIN QUESTION
name: q2_6
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

It turns out the distribution of waiting times in the population follows a very special distribution called the Poisson distribution! Named after French mathematician Baron Siméon Denis Poisson, this distribution has an equal mean and variance. If you go onto to future data science/probability/statistics classes, you will surely see this distribution.

## 3. Computing Intervals

#### Question 3.1
Compute an interval that covers the middle 95% of the **var-based bootstrap estimates**.  Assign your values to `left_end_1` and `right_end_1`. 

*Hint:* Use the `percentile` function! Read up on its documentation [here](http://data8.org/fa20/python-reference.html).

Verify that your interval looks like it covers 95% of the area in the histogram. The red dot on the histogram is the value of the parameter (8).

<!--
BEGIN QUESTION
name: q3_1
manual: false
-->

In [ ]:
left_end_1 = ...
right_end_1 = ...
print("Middle 95% of bootstrap estimates: [{:f}, {:f}]".format(left_end_1, right_end_1))

# Don't change the code below! It draws your interval and V on the histogram of var-based estimates.
import matplotlib.pyplot as plt
Table().with_column("var-based estimate", bootstrap_var_based_estimates).hist(bins=10) 
plt.plot(make_array(left_end_1, right_end_1), make_array(0, 0), color='yellow', lw=5, zorder=1)
plt.scatter(8, 0, color='red', s=30, zorder=2);

#### Question 3.2
Write code that simulates the sampling and bootstrapping process again, as follows:

1. Generate a new set of random observations that Siméon might have seen by sampling from the `population` table we have created for you below. Your sample should have the same number of rows as the original `observations` sample.
2. Compute an estimate of `V` from these new observations, using `var_based_estimator`.
3. Using only the new observations, compute 10,000 bootstrap estimates of `V`. You can use a function you previously defined to do this.
4. Plot these bootstrap estimates (this line of code is given to you) and compute an interval covering the middle 95%.

*Note:* Traditionally, when we bootstrap using a sample from the population, that sample is usually a simple random sample (i.e., sampled uniformly at random from the population without replacement). However, if the population size is big enough, the difference between sampling with replacement and without replacement is negligible. Think about why that's the case! This is why when we define `new_observations`, we sample with replacement.

<!--
BEGIN QUESTION
name: q3_2
manual: false
-->

In [ ]:
# DO NOT CHANGE THESE LINES
from scipy.stats import poisson
pop = Table().with_column("waiting times", poisson.rvs(mu=8, size=100000))
# YOUR CODE BELOW HERE

new_observations = ...
new_var_based_estimate = ...
new_bootstrap_estimates = ...
Table().with_column("var-based estimate", new_bootstrap_estimates).hist(bins=10)
new_left_end = ...
new_right_end = ...

# Don't change code below this line!
print("New var-based estimate: {:f}".format(new_var_based_estimate))
print("Middle 95% of bootstrap estimates: [{:f}, {:f}]".format(new_left_end, new_right_end))

plt.plot(make_array(new_left_end, new_right_end), make_array(0, 0), color='yellow', lw=5, zorder=1)
plt.scatter(8, 0, color='red', s=30, zorder=2);

#### Question 3.3
Does the interval covering the middle 95% of the new bootstrap estimates include `V`?  If you ran that cell 1000 times and generated 1000 intervals, how many of those intervals would you expect to include `V`? 

<!--
BEGIN QUESTION
name: q3_3
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

Let's look at what happens when we use a small number of resamples:

<img src="smallrepvarhist.png" width="525"/>

This histogram and confidence interval was generated using 10 resamples of `new_observations`.

#### Question 3.4
In the cell below, explain why this histogram and confidence interval look different from the ones you generated previously in Question 3.2 where the number of resamples was 10,000.

<!--
BEGIN QUESTION
name: q3_4
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

## 4. The CLT and Book Reviews

Your friend has recommended you a book, so you look for it on an online marketplace. You decide to look at reviews for the book just to be sure that it's worth buying. Let's say that on Amazon, the book only has 80% positive reviews. On GoodReads, it has 95% positive reviews. You decide to investigate a bit further by looking at the percentage of positive reviews for the book on 5 different websites that you know of, and you collect these positive review percentages in a table called `reviews.csv`.

Here, we've loaded in the table for you.

In [ ]:
reviews = Table.read_table("reviews.csv") 
reviews

**Question 4.1**. Calculate the average percentage of positive reviews from your sample and assign it to `initial_sample_mean`.

<!--
BEGIN QUESTION
name: q4_1
manual: false
-->

In [ ]:
initial_sample_mean = ...
initial_sample_mean

You've calculated the average percentage of positive reviews from your sample, so now you want to do some inference using this information. 

**Question 4.2**. First, simulate 5000 bootstrap resamples of the positive review percentages. For each bootstrap resample, calculate the resample mean and store the resampled means in an array called `resample_positive_percentages`. Then, plot a histogram of the resampled means.

<!--
BEGIN QUESTION
name: q4_2
manual: false
-->

In [ ]:
resample_positive_percentages = make_array()

for i in np.arange(5000):
    resample = ...
    resample_avg_positive = ...
    resample_positive_percentages = ...
    
# Do NOT change these lines.
(Table().with_column("Average % of Positive Reviews in Resamples", 
                     resample_positive_percentages).hist("Average % of Positive Reviews in Resamples"))

**Question 4.3**. What is the the shape of the empirical distribution of the average percentage of positive reviews based on our original sample? What value is the distribution centered at? Assign your answer to the variable `initial_sample_mean_distribution`--your answer should be either `1`, `2`, `3`, or `4` corresponding to the following choices:

*Hint: Look at the histogram you made in Question 4.2. Run the cell that generated the histogram a few times to check your intuition.*

1. The distribution is approximately normal because of the Central Limit Theorem, and it is centered at the original sample mean.
2. The distribution is not necessarily normal because the Central Limit Theorem may not apply, and it is centered at the original sample mean.
3. The distribution is approximately normal because of the Central Limit Theorem, but it is not centered at the original sample mean.
4. The distribution is not necessarily normal because the Central Limit Theorem may not apply, and it is not centered at the original sample mean.

<!--
BEGIN QUESTION
name: q4_4
manual: false
-->

In [ ]:
initial_sample_mean_distribution = ...

Though you have an estimate of the true percentage of positive reviews (the sample mean), you want to measure how variable this estimate is. 

**Question 4.4**. Find the standard deviation of your resampled average positive review percentages, which you stored in `resample_positive_percentages`, and assign the result to the variable `resampled_means_variability`.

A convenient way to calculate standard deviation is found [here](https://www.inferentialthinking.com/chapters/14/2/Variability.html#Standard-Deviation) in the textbook. You can also read up on how it relates to variance.

<!--
BEGIN QUESTION
name: q4_5
manual: false
-->

In [ ]:
resampled_means_variability = ...
resampled_means_variability

## 5. Submission


To submit your assignment, please download your notebook as a .ipynb file and submit to Canvas. You can do so by navigating to the toolbar at the top of this page, clicking File > Download as... > Notebook (.ipynb) or HTML (.html). Then, upload both files under "Assignment 5" on Canvas.